In [1]:
import pandas as pd
import geopandas as gpd
import folium
from branca.colormap import linear

# Read the DataFrame
df = pd.read_csv('swiss_no_outliers_and_cleaned.csv')

# Create a GeoDataFrame from the DataFrame
gdf = gpd.GeoDataFrame(df, geometry=gpd.points_from_xy(df.Address_Longitude, df.Address_Latitude))

# Load GeoJSON data into a GeoDataFrame
areas = gpd.read_file('GEN_A4_GEMEINDEN_2019_epsg4326.json')

# Perform spatial join
joined = gpd.sjoin(gdf, areas, how='inner', op='within')

# Calculate average rent for each area
joined['Rent'] = joined['Rent'].astype(float)  # Ensure 'Rent' is float type
area_avg_rent = joined.groupby('NAME')['Rent'].mean()

# Create a map centered at Zurich
map_zurich = folium.Map(location=[47.3769, 8.5417], zoom_start=12)

# Create a colormap for the choropleth
colormap = linear.YlOrRd_09.scale(area_avg_rent.min(), area_avg_rent.max())

# Add the color legend to the map
colormap.add_to(map_zurich)

# Convert the area_avg_rent Series to a DataFrame for easier merging
df_area_avg_rent = area_avg_rent.reset_index()

# Merge the areas GeoDataFrame with the average rent DataFrame
merged = areas.merge(df_area_avg_rent, how='left', left_on='NAME', right_on='NAME')

# Create the choropleth map
folium.Choropleth(
    geo_data=merged,
    name='choropleth',
    data=merged,
    columns=['NAME', 'Rent'],
    key_on='feature.properties.NAME',
    fill_color='YlOrRd',
    fill_opacity=0.7,
    line_opacity=0.2,
    nan_fill_color='white',  # Color for areas with no data
    legend_name='Average Rent',
).add_to(map_zurich)

map_zurich.save("zurich_map.html")


c:\Users\remy\AppData\Local\Programs\Python\Python311\Lib\site-packages\IPython\core\interactiveshell.py:3445: FutureWarning: The `op` parameter is deprecated and will be removed in a future release. Please use the `predicate` parameter instead.
  if await self.run_code(code, result, async_=asy):
C:\Users\remy\AppData\Local\Temp\ipykernel_2816\4000431899.py:16: UserWarning: CRS mismatch between the CRS of left geometries and the CRS of right geometries.
Use `to_crs()` to reproject one of the input geometries to match the CRS of the other.

Left CRS: None
Right CRS: EPSG:4326

  joined = gpd.sjoin(gdf, areas, how='inner', op='within')
